In [1]:
!pip install keras_tuner -q

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def build_model(hp):
    units=hp.Int(name='units',min_value=16,max_value=64,step=16)
    model=keras.Sequential([layers.Dense(units,activation='relu'),
                            layers.Dense(10,activation='softmax')])
    optimizer=hp.Choice(name='optimizer',values=['adagrad','adam'])
    model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model

# Keras Tuner - HyperModel

In [4]:
import keras_tuner as kt
class SimpleMLP(kt.HyperModel):
    def __init__(self,num_classes):
        self.num_classes=num_classes
    def build(self,hp):
        units=hp.Int(name='units',min_value=16,max_value=64,step=16)
        model=keras.Sequential([layers.Dense(units,activation='relu'),
                                layers.Dense(self.num_classes,activation='softmax')])
        optimizer=hp.Choice(name='optimizer',values=['adagrad','adam'])
        model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
        return model
hypermodel=SimpleMLP(num_classes=10)

In [5]:
tuner=kt.BayesianOptimization(build_model,objective=
                              'val_accuracy',max_trials=5,)

In [6]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adagrad', 'conditions': [], 'values': ['adagrad', 'adam'], 'ordered': False}


# Model

In [ ]:
(xtrain,ytrain),(xtest,ytest)=keras.datasets.mnist.load_data()
xtrain=xtrain.reshape((-1,28*28)).astype('float32')/255
xtest=xtest.reshape((-1,28*28)).astype('float32')/255
xtrain_full=xtrain[:]
ytrain_full=ytrain[:]
num_val_samples=10000
xtrain,xval=xtrain[:-num_val_samples],xtrain[-num_val_samples:]
ytrain,yval=ytrain[:-num_val_samples],ytrain[-num_val_samples:]
callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=3),]
tuner.search(xtrain,ytrain,batch_size=128,epochs=20,validation_data=(xval,yval),
            callbacks=callbacks,verbose=2)